# Bootstrapping Hazard Rates

## Import Libraries


In [19]:
#Libraries
import math
import datetime as dt
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import dateutil.relativedelta as relativedelta
from typing import List
import scipy.optimize
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#Import RiVaPy
import rivapy
from rivapy.marketdata import DiscountCurve, DatedCurve, SurvivalCurve
from rivapy import enums

## Credit Curve Bootstrapping Methodology


O'Kane and Turnbull describing the calibration of hazard rates to CDS. Therefore one need to define a standard Credit Defaul Swap. In this CDS one side pays a premium to receive protection for the underlying asset (premium leg). The premium is payed with a certain frequency until the CDS matures or a default occurs. The other side in the CDS exchanges the recovery value in case a default occurs (protection leg).

The valuation and derivation of the premium leg and the protection leg are described in detail in the Credit Defaul Swap valuation Notebook. (Link zu anderem Notebook) In order to valuate both legs a Survival Curve is needed.  

The reduced-form model that we use here is based on the work of Jarrow and Turnbull (1995), who characterize a credit event as the first event of a Poisson counting process which occurs at some time t with a probability defined as :

Pr[τ<t+dt|τ≥t]=λ(t)dt 

the probability of a default occurring within the time interval [t,t+dt) conditional on surviving to time t, is proportional to some time dependent function λ(t), known as the hazard rate, and the length of the time interval dt. We make the simplifying assumption that the hazard rate process is deterministic. By extension, this assumption also implies that the hazard rate is independent of interest rates and recovery rates. From this definition, we can calculate the continuous time survival probability to the time T conditional on surviving to the valuation time tV by considering the limit →0. It can be shown that the survival probability is given by:

Q(tV,T)=exp(−∫TtVλ(s)ds)

Finally, we assume that the hazard rate function is a step-wise constant function. In the below example, the hazard rate between time 0 and 1Y is h0,1=1% and the hazard rate between between 1Y and 3Y is h1,3=2.5%. Therefore we have the 1Y survival probability Q0,1=exp(−h0,1×1)=99% and the 3Y survival probability Q1,3=Q0,1∗exp(−h1,3×2)=91.9%.



In [41]:
h_0_1=0.01
h_0_2=0.025
h_0_3=0.03
h_0_4=0.04

q_0_1=math.exp(-h_0_1*1)
q_1_3=q_0_1*math.exp(-h_0_2*3)
q_1_5=q_1_3*math.exp(-h_0_3*5)
q_1_7=q_1_5*math.exp(-h_0_4*7)

print(q_0_1,q_1_3,q_1_5,q_1_7)


0.9900498337491681 0.9185122844014574 0.7905708496287356 0.5975005946182375


In [56]:
RR=0.4
sp_1=0.0003
sp_2=0.0009
sp_3=0.0015
df1=1
df2=1
df3=1
q_t0=1
q_t1=(q_t0*((1-RR)-sp_1*df1*0.5))/(sp_1*df1*0.5+(1-RR))
print(q_t1)

0.9995001249687578


In [57]:
q_t2=((1-RR)*(q_t0-q_t1)-sp_2*df1*(q_t0+q_t1)*0.5-sp_2*df2*0.5*q_t1+(1-RR)*q_t1)/(sp_2*df2*0.5+(1-RR))
print(q_t2)

0.9970029975643735


In [61]:
q_t3=((1-RR)*(q_t0-q_t1)+(1-RR)*(q_t1-q_t2)+(1-RR)*q_t2-sp_3*df3*0.5*(q_t0+q_t1)-sp_3*df3*0.5*(q_t1+q_t2)-sp_3*df3*0.5*q_t2)/(sp_3*df3*0.5+(1-RR))
print(q_t3)

0.9925180945754479


### Calibration of the Credit Curve

Calibration of the model imply finding an hazard rate (non-cumulative hazard rate) function that matches the market CDS spreads. We make the following assumptions:

- Swap premium payments are made quarterly following a business day calendar
- Hazard rate is a piece-wise constant function of time (i.e. hazard rates are independent from interest rates)
- Recovery rate is constant

The construction of the hazard rate term structure is done by an iterative process called bootstrapping. Let’s assume we have quotes for 1Y, 3Y, 5Y and 7Y for a given issuer. From the 1Y CDS spread s1Y, we will find the hazard rate λ0,1 which equates the present value of the premium leg and of the protection leg. From the 3Y spread s3Y, we will find the hazard rate λ1,3 which equates the present value of the premium leg to the present value of the protection leg. By iterating this process, we obtain the hazard rates: λ0,1,λ1,3,λ3,5,λ5,7. We can also easily calculate the survival probabilities from this hazard rate term structure (as we have seen earlier).

In [42]:
# Beispiel Rechnung

## Calculate Hazard Rates for specific Market Data

### Define Functions

In [7]:
def par_spread(ref_date, trade_date, dc_survival, payment_dates, maturity_date, dc, RR):
    integration_step= relativedelta.relativedelta(days=365)
    premium_period_start = ref_date
    prev_date=ref_date
    current_date=min(prev_date+integration_step, maturity_date)
    dc_valuation_date=dc.value(ref_date, maturity_date)
    risk_adj_factor_protection=0
    risk_adj_factor_premium=0
    risk_adj_factor_accrued=0
    #RR=0.4

    while current_date <= maturity_date:
        default_prob = dc_survival.value(ref_date, prev_date)-dc_survival.value(ref_date, current_date)
        #print("default_prob:",default_prob)
        risk_adj_factor_protection += dc.value(ref_date, current_date) * default_prob
        #print("risk_adj_factor_protection:",risk_adj_factor_protection)
        prev_date = current_date
        current_date += integration_step
    
    if prev_date < maturity_date and current_date > maturity_date:
        default_prob = dc_survival.value(ref_date, prev_date)-dc_survival.value(ref_date, maturity_date)
        #print("default_prob:",default_prob)
        risk_adj_factor_protection += dc.value(ref_date, maturity_date)  * default_prob
        #print("risk_adj_factor_protection:",risk_adj_factor_protection)


    for premium_payment in payment_dates:
        if premium_payment >= ref_date:
            period_length = ((premium_payment-premium_period_start).days)/360
            #survival_prob = (dc_survival.value(ref_date, premium_period_start)+dc_survival.value(ref_date, premium_payment))/2
            survival_prob = dc_survival.value(ref_date, premium_period_start)
            #print("survival_prob:",survival_prob)
            df = dc.value(ref_date, premium_payment)
            risk_adj_factor_premium += period_length*survival_prob*df
            #print("risk_adj_factor_premium:",risk_adj_factor_premium)
            default_prob = dc_survival.value(ref_date, premium_period_start)-dc_survival.value(ref_date, premium_payment)
            #print("default_prob_accrued:",default_prob)
            risk_adj_factor_accrued += period_length*default_prob*df
            #print("risk_adj_factor_accrued:",risk_adj_factor_accrued)
            premium_period_start = premium_payment



    PV_accrued=((1/2)*risk_adj_factor_accrued)
    #print("PV_accrued: ",PV_accrued)
    PV_premium=(1)*risk_adj_factor_premium
    #print("PV_premium: ",PV_premium)
    PV_protection=(((1-RR))*risk_adj_factor_protection)
    #print("PV_protection: ",PV_protection)
    
    par_spread_i=(PV_protection)/((PV_premium+PV_accrued))
    #print("par_spread_i: ",par_spread_i)
    return par_spread_i

In [8]:
#Function to Rollout payment_dates
def payment_dates(trade_date,maturity_date,payment_cycle):
    date=maturity_date-relativedelta.relativedelta(months=+payment_cycle)
    payment_dates=[maturity_date]
    if maturity_date<date:
       payment_dates.append(maturity_date)
    while date>=trade_date:
        payment_dates.append(date)
        date=date-relativedelta.relativedelta(months=+payment_cycle)
    return sorted(payment_dates)
#payment_dates_ins=payment_dates(trade_date,maturity_date,payment_cycle)

In [9]:
#Bootstrapping Functionality
def create_survival(refdate, dates: List[dt.datetime], hazard_rates: List[float]):
    return SurvivalCurve('survival_curve', refdate, dates, hazard_rates)
    
def calibration_error(x, mkt_par_spread, ref_date, trade_date, payment_dates, dc_new, dates, hazard_rates, RR):
    hazard_rates[-1] = x
    maturity_date = dates[-1]
    dc_surv = create_survival(ref_date, dates, hazard_rates)
    #print(x)
    #print("Market Spread=",mkt_par_spread,"-","Par-Spread=",par_spread(ref_date, trade_date, dc_surv, payment_dates,maturity_date, dc_new),"=",
    #      mkt_par_spread - par_spread(ref_date, trade_date, dc_surv, payment_dates,maturity_date, dc_new))
    return  mkt_par_spread - par_spread(ref_date, trade_date, dc_surv, payment_dates, maturity_date, dc_new, RR)


def calibrate_hazard_rate(payment_dates_bootstrapp, ref_date,mkt_par_spread, trade_date, dc_new, RR):
    #sort payment_dates by [-1]
    sc_dates=[ref_date]
    hazard_rates=[0.0]
    for i in range(len(payment_dates_bootstrapp)):
        payment_dates_iter = payment_dates_bootstrapp[i]
        #print(payment_dates_iter)
        mkt_par_spread_iter = mkt_par_spread[i]
        #print(mkt_par_spread_iter)
        sc_dates.append(payment_dates_iter[-1])
        hazard_rates.append(hazard_rates[-1])
        sol=scipy.optimize.root_scalar(calibration_error,args=(mkt_par_spread_iter, ref_date, trade_date, 
                         payment_dates_iter, dc_new, sc_dates, hazard_rates, RR),method='brentq',bracket=[0,3],xtol=1e-8,rtol=1e-8)
        hazard_rates[-1] = sol.root
        #print(numpy.exp(-sol.root*((payment_dates_iter[-1]-ref_date).days/365)))
    return create_survival(ref_date, sc_dates, hazard_rates), hazard_rates



### Define market data

In [34]:
#Define new yield curve
#yield curve
object_id = "CDS_interest_rate_boots"
refdate = dt.datetime(2020,1,1)
days_to_maturity = [360, 720, 3*360, 4*360, 5*360, 7*360, 10*360,  15*360, 20*360,  30*360]
dates = [refdate + timedelta(days=d) for d in days_to_maturity]
rates = [0.002585,0.005034,0.008981,0.012954,0.016452,
                   0.021811,0.027007,0.031718,0.033834,0.035056]
#rates = [0,0,0,0,0,0,0,0,0,0]
dsc_fac = [math.exp(-rates[i]*days_to_maturity[i]/360) for i in range(len(days_to_maturity))]

        
dc_new = DiscountCurve(object_id, refdate, dates, dsc_fac, enums.InterpolationType.LINEAR, enums.ExtrapolationType.LINEAR)

In [35]:
#Market Spreads
#Pfizer
mkt_par_spread=[0.0003,0.0009,0.0015,0.0021,0.0028,0.0043,0.0061,0.0063, 0.0068,0.0066]

#Radioshack
#mkt_par_spread=[0.6405,0.5956,0.5511,0.5144,0.4894,0.4511,0.4156,0.3815,0.3657,0.3506]

### Define Instrument data

In [36]:
#Instrument data
refdate = dt.datetime(2020,1,1)
trade_date = refdate
RR=0.4
maturities_CDS_test = [12,24,36,48,60,7*12,10*12,15*12,20*12,30*12]

#Maturities and Payment Dates
#ref_date=dt.datetime(2020,1,1)
maturity_dates=[]
date=refdate
for i in range (len(maturities_CDS_test)):
    if maturities_CDS_test[i]>=12:
        date=refdate+relativedelta.relativedelta(years=(maturities_CDS_test[i]/12))
        maturity_dates.append(date)
    elif maturities_CDS_test[i]<12:
        date=refdate+relativedelta.relativedelta(month=maturities_CDS_test[i])
        maturity_dates.append(date)
    
maturity_dates_sorted=sorted(maturity_dates)

#Rollout instrument payment dates for different maturities
payment_dates_bootstrapp=[]
for i  in range(len(maturity_dates_sorted)):
    payment_dates_bootstrapp.append(payment_dates(refdate,maturity_dates_sorted[i],3))
payment_dates_bootstrapp=sorted(payment_dates_bootstrapp, key=lambda x: x[-1])


### Bootstrapp Survival Curve

In [37]:
#Create Survival Curve based on Bootstrapping alogirthm
survival_curve=calibrate_hazard_rate(payment_dates_bootstrapp, refdate,mkt_par_spread, trade_date, dc_new, RR)
survival_curve=survival_curve[0]
survival_curve_bootstrap=[survival_curve.value(refdate,refdate)]

for i in range(len(payment_dates_bootstrapp)):
    survival_curve_bootstrap.append(survival_curve.value(refdate,payment_dates_bootstrapp[i][-1]))
del survival_curve_bootstrap[0]
print(survival_curve_bootstrap)

[0.9994912093001219, 0.9969419228330689, 0.9923236473576058, 0.9856001331543911, 0.9758396345714148, 0.9475631162999186, 0.8933873221849499, 0.841921965002223, 0.7764472038292964, 0.7038634891826675]


### Get Data for Plots

In [28]:
#Hazard Rates for plot
hazard_rates=calibrate_hazard_rate(payment_dates_bootstrapp, refdate,mkt_par_spread, trade_date, dc_new, RR)[1]
del hazard_rates[0]
hazard_rates=[rate*100 for rate in hazard_rates]
#print(hazard_rates)
#Survival Rates for plot
survival_rates=survival_curve_bootstrap
survival_rates=[rate*100 for rate in survival_rates]
#print(survival_rates)

### Plot Data

In [27]:
#Plotting Hazard Rates for different maturities
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=x, y=survival_rates, name="Survival Rate (in percentage)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=x, y=hazard_rates, name="Hazard Rate (in percentage)"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hazard Rates Bootstrapped"
)

# Set x-axis title
fig.update_xaxes(title_text="Maturity (in months)")

# Set y-axes titles
fig.update_yaxes(title_text="Survival Rate (in %)", secondary_y=False)
fig.update_yaxes(title_text="Hazard Rate (in %)", secondary_y=True)

fig.show()